## SEARCH ALGORITHMS

in this project, we are going to make state grapgh and do BFS, IDS and A* search on it, analyzing the result and say pros and cons of each algorithms

In [783]:
from copy import deepcopy,copy
from bisect import insort_left
from timeit import default_timer as timer
import heapq

In [784]:
IDS_mode = 1
BFS_mode = 2

In [785]:
nodes = dict()#global
loose_nodes_revival = dict()#global
ziraj = 0

pizzas = dict()#unique
priorities = set()#unique
looses_traverse_init = dict()#unique

edges = []#temp
students = []#temp

In [786]:
def read_input(file_id):
    global nodes,loose_nodes_revival,pizzas,priorities,looses_traverse_init,edges,students,ziraj
    nodes = dict()
    loose_nodes_revival = dict()
    pizzas = dict()
    priorities = set()
    looses_traverse_init = dict()
    edges = []
    students = []
    
    f = open(f'test{file_id}.txt',mode='r')
    
    n_nodes, n_edges = [int(i) for i in f.readline().split()]

    for i in range(n_edges):
        node1,node2 = [int(j) for j in f.readline().split()]
        edges.append((min(node1,node2),max(node1,node2)))

        if node1 in nodes.keys():
            nodes[node1].append(node2)
        else:
            nodes[node1] = [node2]
        if node2 in nodes.keys():
            nodes[node2].append(node1)
        else:
            nodes[node2] = [node1]

    n_loose_edges = int(f.readline())
    for i in range(n_loose_edges):
        edge_index, revival_time = [int(j) for j in f.readline().split()]
        loose_nodes_revival[edges[edge_index - 1]] = revival_time

    max_revival = max(loose_nodes_revival.values())
    for i in loose_nodes_revival.keys():
        looses_traverse_init[i] = -1 * max_revival
    
    ziraj = int(f.readline())
    n_students = int(f.readline())
    for i in range(n_students):
        student_node, pizza_node = [int(j) for j in f.readline().split()]
        students.append(student_node)
        pizzas[pizza_node] = student_node

    n_order = int(f.readline())
    for i in range(n_order):
        prior, after = [int(j) for j in f.readline().split()]
        priorities.add((students[prior - 1], students[after - 1]))

to solve this problem, I consider every graph and its all information such as Ziraj's position, delivered pizza and ... as a state, every movement of ziraj is considered as a new state, each two states which only differs in a single movement(moving to a aside node or taking a pizza to deliver or giving pizza to a student) has an edge to each other, so we do search algorithm's on state graph.

In [800]:
class State:

    def __init__(self, _time, _node, _looses, _way, _curr_pizza, _priority, _pizzas, _explored, _abandoned):
        self.time = _time #int showing time 
        self.curr_node = _node#showing which node we are in
        self.looses_last_traverse = _looses#dict of tuple of loose edges
        self.way = _way#list of traversed nodes
        self.curr_pizza = _curr_pizza#set of which pizza is being delivered
        self.priority = _priority#set of tuple :(first,then)
        self.pizzas_to_deliver = _pizzas#dict of pizzas,dict[pizza]=student
        self.explored = _explored#set of explored states
        self.abandoned = _abandoned#if we wait for a loose edge, other nodes connected to that specific node expect loose one

    def __key(self):
        return(self.curr_node,self.curr_pizza,self.pizzas_to_deliver)

    def __str__(self):
        return(str(self.__key()))

    def __lt__(self,other):
        return self.heuristic() < other.heuristic()
        
    def can_be_delivered(self):
        if len(self.curr_pizza) == 0:
            return False,-1

        for pizza in self.curr_pizza:
            if self.pizzas_to_deliver[pizza] == self.curr_node:
                return True,pizza

        return False,-1

    def is_finished(self):
        return len(self.pizzas_to_deliver) == 0
    
    def heuristic(self, weight = 1):
        return self.time + weight * 2 * (len(self.pizzas_to_deliver))
        
    def update_explored(self):
        children = set()
        for node in self.explored:
            children.update(nodes[node])
        self.explored.update(children)

ziraj can do this movements:
1. stay in a node for a loose edge which can not be passed.
2. move to a new node and grab a pizza if the pizza can be taken
3. move to a new node and if he can give the pizza to its owner, he must deliver it
4. just move to a new node

if we reach a node with pizza on it, and there is not a prior pizza before it, he either can take it or not, so we have two seperate states, to avoid making 2 states(if we do this, number of states grow exponentially $2^{pizzas}$), we make a list which we considered we have taken every pizza on our way, when we can deliver a existing pizza in list to its owner, we make two states, delivering that pizza and removing all pizzas in list or ignoring that owner(!) to deliver other pizzas in list, with this impeliment we make much less states. 

In [788]:
def move(prev_state, next_node, is_loose, edge, search_type):

    new_state = deepcopy(prev_state)
    new_state.curr_node = next_node
    new_state.time += 1
    new_state.way = new_state.way + [next_node]

    if search_type == IDS_mode:
         new_state.update_explored()
    elif search_type == BFS_mode:
        new_state.explored.add(next_node)

   
    if(is_loose):
        new_state.looses_last_traverse[edge] = new_state.time
        new_state.abandoned.clear()

    delivery,pizza = new_state.can_be_delivered()
    if(delivery):
        new_state_delivery = deepcopy(new_state)
        del new_state_delivery.pizzas_to_deliver[pizza]
        new_state_delivery.curr_pizza.clear()
        new_state_delivery.explored = {next_node}
        new_state_delivery.priority = set([x for x in new_state.priority if x[0] != new_state.curr_node])
        return new_state, new_state_delivery

    elif new_state.curr_node in new_state.pizzas_to_deliver.keys() and new_state.curr_node not in new_state.curr_pizza:
        if not any(x[1] == new_state.pizzas_to_deliver[new_state.curr_node] for x in new_state.priority):
            new_state.curr_pizza.add(new_state.curr_node)
            new_state.explored = {next_node}

    return new_state, None


def wait(curr_state,other_nodes):
    new_state = deepcopy(curr_state)
    new_state.time += 1
    new_state.way = new_state.way + [new_state.curr_node]
    new_state.abandoned.update(other_nodes)
    return new_state

## BFS

on BFS search, we add all connected states to current state in state graph to a queue and continue our search in state graph to reach a state which all pizzas are delivered, so we found answer and optimal way.

to keep BFS optimal, we must consider every waiting on a node for a loose edge as a new seperated state.

In [789]:
def BFS():
    start = State(0,ziraj,looses_traverse_init,[ziraj],set(),priorities,pizzas,{ziraj},set())
    queue = [start]
    checked = 1
    explored = {str(start)}
    while(queue):

        visiting = queue.pop(0)
        checked += 1
        for frontier in nodes[visiting.curr_node]:
            if frontier in visiting.explored or frontier in visiting.abandoned:
                continue
            edge = (min(frontier,visiting.curr_node),max(frontier,visiting.curr_node)) 
            is_loose = False
            if edge in loose_nodes_revival.keys():
                waiting_time = loose_nodes_revival[edge] - (visiting.time - visiting.looses_last_traverse[edge])
                if waiting_time > 0:
                    other_nodes = set(x for x in nodes[visiting.curr_node] if x != frontier)
                    new_state = wait(visiting,other_nodes)
                    queue.append(new_state)
                    continue
                is_loose = True

            new_state, new_state_delivery = move(visiting,frontier,is_loose,edge,BFS_mode)

            if(new_state_delivery != None):

                if(new_state_delivery.is_finished()):
                    return checked, new_state_delivery.way
                if(str(new_state_delivery) not in explored):
                    explored.add(str(new_state_delivery))
                    queue.append(new_state_delivery)

            if(str(new_state) not in explored):
                    explored.add(str(new_state))
                    queue.append(new_state)
            

In [790]:
for i in range(1,4):
    read_input(i)
    time = 0
    for j in range(3):
        begin = timer()
        total_states, way = BFS()
        end = timer()
        time += (end - begin)
    print(f'test {i} :')
    print(f'total states seen = {total_states}')
    print(f'answer = {len(way)}')
    for k in range(len(way) - 1):
        print(way[k],end=' -> ')
    print(way[-1])
    print(f'average time = {time / 3} seconds')
    print()

test 1 :
total states seen = 227
answer = 11
6 -> 4 -> 10 -> 5 -> 3 -> 8 -> 9 -> 1 -> 7 -> 9 -> 2
average time = 0.03375596666834705 seconds

test 2 :
total states seen = 684
answer = 21
14 -> 9 -> 4 -> 15 -> 12 -> 3 -> 12 -> 6 -> 14 -> 1 -> 14 -> 12 -> 3 -> 8 -> 6 -> 4 -> 10 -> 11 -> 8 -> 3 -> 13
average time = 0.1369244333327515 seconds

test 3 :
total states seen = 27773
answer = 32
15 -> 14 -> 8 -> 20 -> 13 -> 8 -> 18 -> 10 -> 8 -> 9 -> 16 -> 5 -> 15 -> 12 -> 3 -> 19 -> 11 -> 19 -> 13 -> 6 -> 13 -> 1 -> 13 -> 8 -> 17 -> 8 -> 18 -> 2 -> 18 -> 4 -> 14 -> 16
average time = 6.8163256666642456 seconds



## IDS
we put a limit for DFS, if depth becomes more than limit and we have not found answer we return None and increase limit and do another DFS, we repeat this to finally find answer, it is obvious that this method returns optimal answer.

to make IDS faster we do this:

consider we are in depth n, we abandon all n-1 depths' nodes which has same information(pizzas to deliver, and delivered pizzas) and we do not pass the along the way, the reason is if they are involved in answer, they will be iterated with another way, which pass through one of our nodes' uncle with a shorter way.

In [791]:
def IDS(curr_state,limit,states):
    if curr_state.is_finished():
        return curr_state

    depth = len(curr_state.way) - 1
    if(depth > limit):
        return None

    children = []
    for frontier in nodes[curr_state.curr_node]:
        if frontier in curr_state.explored or frontier in curr_state.abandoned:
            continue
        edge = (min(frontier,curr_state.curr_node),max(frontier,curr_state.curr_node)) 
        is_loose = False
        if edge in loose_nodes_revival.keys():
            waiting_time = loose_nodes_revival[edge] - (curr_state.time - curr_state.looses_last_traverse[edge])
            if waiting_time > 0:
                other_nodes = set(x for x in nodes[curr_state.curr_node] if x != frontier)
                new_state = wait(curr_state, other_nodes)
                children.append(new_state)
                continue
            is_loose = True
            
        new_state, new_state_delivery = move(curr_state,frontier,is_loose,edge,IDS_mode)

        if(new_state_delivery != None):

            if(new_state_delivery.is_finished()):
                return new_state_delivery
            
            if(str(new_state_delivery) not in ids_ex):
                children.append(new_state_delivery)
                ids_ex.add(str(new_state_delivery))


        if(str(new_state) not in ids_ex):
            children.append(new_state)
            ids_ex.add(str(new_state))

    for child in children:
        states.append(1)
        ans = IDS(child,limit,states)
        if ans != None:
            return ans

    for child in children:
        if(str(child) in ids_ex):
            ids_ex.remove(str(child))


In [819]:
for m in range(1,3):
    read_input(m)
    first_state = State(0,ziraj,looses_traverse_init,[ziraj],set(),priorities,pizzas,{ziraj},set())
    time = 0
    for k in range(3):
        states = [1]
        depth = 0
        start = timer()
        while True:
            ids_ex = {str(first_state)}
            ans = IDS(first_state, depth, states)
            if ans != None:
                break
            depth += 1
        end = timer()
        time += (end - start)
        
    print(f'test {m} :')
    print(f'total states seen = {len(states)}')
    print(f'answer = {len(ans.way)}')
    for k in range(len(ans.way) - 1):
        print(ans.way[k],end=' -> ')
    print(ans.way[-1])
    print(f'average time = {time / 3} seconds')
    print()
        

test 1 :
total states seen = 3871
answer = 11
6 -> 4 -> 10 -> 5 -> 3 -> 8 -> 9 -> 1 -> 7 -> 9 -> 2
average time = 0.2007790666660488 seconds

test 2 :
total states seen = 220543
answer = 21
14 -> 9 -> 4 -> 15 -> 12 -> 3 -> 12 -> 6 -> 14 -> 1 -> 14 -> 12 -> 3 -> 8 -> 6 -> 4 -> 10 -> 11 -> 8 -> 3 -> 13
average time = 15.653787233333182 seconds



the third test can not be done within 20 minute, as we see growth rate is exponential and its about $2.5^n$, so we must expand about 86000 billion states!

## A*

I consider 2 $\times$ number of remaining pizzas to deliver as heuristic, the minimum time to deliver a pizza is two(one to take, one to deliver, according to the fact that students and pizzas are in different nodes), so heuristic function is consistent therefore admissible

for every a,b node we have:
$$h(a) - h(b) \leq 2 \leq Realcost(a,b)$$

In [793]:
def A_star(weight = 1):
    start = State(0,ziraj,looses_traverse_init,[ziraj],set(),priorities,pizzas,{ziraj},set())
    queue = [(start.heuristic(weight),start)]
    checked = 1
    explored = {str(start)}
    while(queue):
        visiting = heapq.heappop(queue)[1]
        checked += 1
        for frontier in nodes[visiting.curr_node]:

            if frontier in visiting.explored or frontier in visiting.abandoned:
                continue

            edge = (min(frontier,visiting.curr_node),max(frontier,visiting.curr_node)) 
            is_loose = False
            if edge in loose_nodes_revival.keys():
                waiting_time = loose_nodes_revival[edge] - (visiting.time - visiting.looses_last_traverse[edge])
                if waiting_time > 0:
                    other_nodes = set(x for x in nodes[visiting.curr_node] if x != frontier)
                    new_state = wait(visiting,other_nodes)
                    heapq.heappush(queue,(new_state.heuristic(weight),new_state))
                    continue
                is_loose = True

            new_state, new_state_delivery = move(visiting,frontier,is_loose,edge,BFS_mode)

            if(new_state_delivery != None):
                if(new_state_delivery.is_finished()):
                    return checked, new_state_delivery.way
                if(str(new_state_delivery) not in explored):
                    explored.add(str(new_state_delivery))
                    heapq.heappush(queue,(new_state_delivery.heuristic(weight),new_state_delivery))

            if(str(new_state) not in explored):
                    explored.add(str(new_state))
                    heapq.heappush(queue,(new_state.heuristic(weight),new_state))


In [801]:
for i in range(1,4):
    read_input(i)
    time = 0
    for j in range(3):
        begin = timer()
        total_states, way = A_star()
        end = timer()
        time += (end - begin)
    print(f'test {i} :')
    print(f'total states seen = {total_states}')
    print(f'answer = {len(way)}')
    for k in range(len(way) - 1):
        print(way[k],end=' -> ')
    print(way[-1])
    print(f'average time = {time / 3}')
    print()

test 1 :
total states seen = 100
answer = 11
6 -> 4 -> 10 -> 5 -> 3 -> 8 -> 9 -> 1 -> 7 -> 9 -> 2
average time = 0.015586099997259831

test 2 :
total states seen = 542
answer = 21
14 -> 9 -> 4 -> 15 -> 12 -> 3 -> 8 -> 6 -> 14 -> 1 -> 14 -> 12 -> 6 -> 8 -> 6 -> 4 -> 10 -> 11 -> 5 -> 2 -> 13
average time = 0.11510286666695417

test 3 :
total states seen = 24012
answer = 32
15 -> 14 -> 4 -> 20 -> 13 -> 10 -> 18 -> 10 -> 8 -> 9 -> 15 -> 5 -> 16 -> 12 -> 3 -> 19 -> 11 -> 19 -> 13 -> 6 -> 13 -> 1 -> 13 -> 19 -> 17 -> 8 -> 20 -> 2 -> 18 -> 4 -> 14 -> 16
average time = 6.197502766665518



## Weighted A*

Weighted heuristic will decrease execution time so much but we lose optimality

we reach a point which incresing weight won't affect total seen states, in this case we have greedy-best first


In [813]:
weights = [2.2, 4 , 7]
for weight in weights:
    print(f'weight of heuristic is : {weight}, result:\n')
    for i in range(1,4):
        read_input(i)
        time = 0
        for j in range(3):
            begin = timer()
            total_states, way = A_star(weight)
            end = timer()
            time += (end - begin)
        print(f'test {i} :')
        print(f'total states seen = {total_states}')
        print(f'answer = {len(way)}')
        for k in range(len(way) - 1):
            print(way[k],end=' -> ')
        print(way[-1])
        print(f'average time = {time / 3}')
        print()

weight of heuristic is : 2.2, result:

test 1 :
total states seen = 30
answer = 12
6 -> 9 -> 1 -> 4 -> 10 -> 5 -> 3 -> 8 -> 9 -> 7 -> 9 -> 2
average time = 0.006460699999782567

test 2 :
total states seen = 162
answer = 22
14 -> 9 -> 4 -> 15 -> 12 -> 3 -> 8 -> 6 -> 8 -> 6 -> 4 -> 9 -> 14 -> 1 -> 14 -> 12 -> 6 -> 8 -> 11 -> 10 -> 4 -> 13
average time = 0.04447066666762112

test 3 :
total states seen = 858
answer = 34
15 -> 14 -> 4 -> 20 -> 8 -> 20 -> 2 -> 20 -> 13 -> 10 -> 18 -> 10 -> 8 -> 9 -> 16 -> 5 -> 15 -> 12 -> 3 -> 19 -> 11 -> 19 -> 13 -> 6 -> 13 -> 1 -> 13 -> 8 -> 17 -> 5 -> 7 -> 4 -> 14 -> 16
average time = 0.2841305666685609

weight of heuristic is : 4, result:

test 1 :
total states seen = 30
answer = 12
6 -> 9 -> 1 -> 4 -> 10 -> 5 -> 3 -> 8 -> 9 -> 7 -> 9 -> 2
average time = 0.00631249999908808

test 2 :
total states seen = 80
answer = 22
14 -> 9 -> 4 -> 15 -> 12 -> 3 -> 8 -> 6 -> 8 -> 6 -> 4 -> 9 -> 14 -> 1 -> 14 -> 12 -> 6 -> 14 -> 11 -> 5 -> 2 -> 13
average time = 0.03093

|  | answer | Total states seen | Avg Time |
| --- | --- | --- | --- |
| BFS |  |  |  |
| Test 1 | 11 | 227 | 0.03375s |
| Test 2 | 21 | 684 | 0.13692s |
| Test 3 | 32 | 27773 |  6.81632s |
| IDS |  |  |  |
| Test 1 | 11 | 3871 | 0.20077s |
| Test 2 | 21 | 220543 | 15.65378s |
| Test 3 | 32 | about 86000B |  ! |
| A* |  |  |  |
| Test 1 | 11 | 100 | 0.01558s |
| Test 2 | 21 | 542 | 0.11510s |
| Test 3 | 32 | 24012 | 6.19750s |
| A* , weight = 2.2 | |  |
| Test 1 | 12 | 30 | 0.00646s |
| Test 2 | 22 | 162 | 0.04447s |
| Test 3 | 34 | 858 |  0.28413s |
| A*  , weight > 4 | --- | --- |
| Test 1 | 12 | 30 | 0.004499s |
| Test 2 | 22 | 80 | 0.01593s |
| Test 3 | 35 | 181 | 0.044341s  |


* As we see in this table, IDS, BFS and A* result in optimal answer
*  with a consistent heuristic A* is faster and better than others, but there is a trade off between calculating heuristic time and expanding more states time
* IDS is not good for this problem
* weighted A* is amazingly fast, but it is not optimal.
* if weight becomes so big, A* turns into greedy-best first search